In [ ]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras import backend as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding, Dropout, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import model_from_json

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

import pickle

# print(tf.__version__)
# print(tf.keras.__version__)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# This code is based on a tutorial which can be found here:
# https://realpython.com/python-keras-text-classification/#what-is-a-word-embedding

In [ ]:
#Packages to import (regular expressions and pandas)

import re 
import pandas as pd 

from pathlib import Path
import gzip
import json

# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Mount google colab to google drive so it can access files
from google.colab import drive
drive.mount('/content/drive')

#The huge gzipped JSON files ('NL_jobs.15.jsonl.gz') are located in this folder
cwd = '/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments'

#Print the content of the folder
!ls '/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments'

# set current working directory, This folder should only contain the huge json files.
%cd '/content/drive/My Drive/Werkinzicht_Clyde/Notebooks/Experiments'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Baseline Model (BOW Log. regression).ipynb'
 BERT_alt_deploy.ipynb
'BERT_Jads_Nikita- Clyde edit - hpartuning.ipynb'
 BOW_LSTM.ipynb
'Copy of Copy of BERT_alt _ hyperparamtuning.ipynb'
 CORPUSNL_jobs.15.jsonl.gz_JSON_splitted.tsv_1.tsv
 Cow
 COW_LSTM_hyperparamtuning.ipynb
 dev.gsheet
 dev.tsv
 df3_updated_clean.txt
 df3_updated.txt
 df4_updated_clean.txt
 df4_updated.txt
 Exp23_RNDSCV.csv
 Exp25_RNDSCV.csv
 Exp26_RNDSCV.csv
 Exp27_RNDSCV.csv
 Exp28_RNDSCV.csv
 Exp29_RNDSCV.csv
 Exp30_RNDSCV.csv
 Exp31_RNDSCV.csv
 Exp32_RNDSCV.csv
 Exp33_RNDSCV.csv
 Exp34_RNDSCV.csv
 Exp35_RNDSCV.csv
 Exp36_RNDSCV.csv
 fastText-0.1.0
 glove.6B
 glove.6B.zip
 glove.model
 model.h5
 model.json
 NL_Embedding
 NL_jobs.15.jsonl.gz_JSON_splitted.tsv
 temp
 tokenizer.pickle
 train.gsheet
 train.tsv
 v0.1.0.zip
 v0.1.0.zip.1
 v0.1.0.zip.2
 Word2Vec_LSTM_hyperparamtuning.ipynb
 Word_

In [ ]:
TRAIN_FILE = "train.tsv"
DEV_FILE = "dev.tsv"

print("TRAIN FILE: \t\t{}\nDEVELOPMENT FILE: \t{}".format(TRAIN_FILE, DEV_FILE))

TRAIN FILE: 		train.tsv
DEVELOPMENT FILE: 	dev.tsv


In [ ]:
# Load the data
test_full = pd.read_csv(TRAIN_FILE, sep = "\t")
dev_full = pd.read_csv(DEV_FILE, sep = "\t")

# Tirm the data to only one we need
test_trim = test_full[["Quality", "#1 String"]]
dev_trim = dev_full[["Quality", "#1 String"]]

# Seperate our data for training and testing
train_text = np.array(test_full["#1 String"])
train_target = np.array(test_full["Quality"])

test_text = np.array(dev_full["#1 String"])
test_target = np.array(dev_full["Quality"])

# Check our training and testing sets
print("Train-set size: ", len(train_text))
print("Test-set size:  ", len(test_text))

# Stack text for tokenization
data_text = np.hstack((train_text, test_text))
print("Shape of the stacked text: {}".format(data_text.shape))

Train-set size:  2240
Test-set size:   720
Shape of the stacked text: (2960,)


In [ ]:
# Get stop words (or most commoon ones basically) for the dutch language
stop_words = set(stopwords.words('dutch'))

# Get the tokinized pattern of all sentences
word_tokens = []
for sentence in data_text:
    word_tokens.append(nltk.word_tokenize(sentence))

# Initiate a new collector
filtered_sentence = []
for entry in word_tokens:
    filtered_sentence.append([w for w in entry if not w in stop_words and w.isalpha()])
filtered_sentence = np.array(filtered_sentence)

# Implement the top level tokinizer, we will go for 10K words
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(filtered_sentence)

In [ ]:
# tokenizer.word_index # If we need to check it

In [ ]:
# Save the tokinizer for re-usability with the new data
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
x_train_tokens = tokenizer.texts_to_sequences(train_text)
x_test_tokens = tokenizer.texts_to_sequences(test_text)

vocab_size = len(tokenizer.word_index) + 1  

# Example of conversions
print("Conversion Examples:")
print(x_train_tokens[50])
print(train_text[50])
print("---------------------------------")
print(x_train_tokens[100])
print(train_text[100])
print("---------------------------------")

# Padding the data if needed
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
print("Average Number of Tokens per sentence: ", np.round(np.mean(num_tokens), 4))
print("Maxmimum Number of Tokens per sentence: ", np.max(num_tokens))
print("Minimum Number of Tokens per sentence: ", np.min(num_tokens))

# The max number of tokens we will allow is set to the average plus 2 standard deviations.
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
print("Maximum amount of tokens to hold: ", max_tokens)

# This covers almost 96% of the data
print("How much data does the padding include fully: {}".format(np.sum(num_tokens < max_tokens) / len(num_tokens)))

# Initiate the condition for padding
pad = 'pre'

# Applying padding to the training data
x_train_pad = pad_sequences(x_train_tokens,
                            maxlen=max_tokens,
                            padding=pad,
                            truncating=pad)

# Applying padding to the development data

x_test_pad = pad_sequences(x_test_tokens, 
                           maxlen=max_tokens, 
                           padding=pad, 
                           truncating=pad)

# Verify that all of the padding is done correctly
print("Shape of the padded testing data: \t", x_train_pad.shape)
print("Shape of the padded development data: \t", x_test_pad.shape)
# Seems correct

Conversion Examples:
[233, 2386, 245, 2387]
Utrecht, Drenthe en Flevoland.
---------------------------------
[13, 152, 614, 298, 1211, 2416, 298, 2417, 462, 237, 298, 2418, 835, 115, 823, 2419, 710]
het, met behulp van ons calculatieprogramma BV6 van TSD uit Zwolle, uitwerken van prijsaanvragen tot een correcte werkomschrijvende offerte
---------------------------------
Average Number of Tokens per sentence:  12.7966
Maxmimum Number of Tokens per sentence:  58
Minimum Number of Tokens per sentence:  0
Maximum amount of tokens to hold:  29
How much data does the padding include fully: 0.9618243243243243
Shape of the padded testing data: 	 (2240, 29)
Shape of the padded development data: 	 (720, 29)


In [ ]:
# def create_model(lstm1, lstm2, dense1, vocab_size, embedding_dim, maxlen):
#     #Creating the network
#     # embedding_size = 256
#     # num_words = num_words # We chose this value earlier (10,000)
#     # max_tokens = max_tokens # This values is also chosen from before (30)

#     # For reproducability setting a seed
#     seed = 42
#     # np.random.seed(seed)

#     model = Sequential()
#     model.add(Embedding(input_dim=vocab_size,
#                         output_dim=embedding_dim,
#                         input_length=maxlen,
#                         name='layer_embedding'))
#     model.add(Bidirectional(LSTM(units = 32, return_sequences=True, name = "BiLSTM_1")))
#     model.add(Dropout(0.5, noise_shape=None, seed=seed))
#     model.add(Bidirectional(LSTM(units = 16, return_sequences=False, name = "BiLSTM_2")))
#     model.add(Dropout(0.5, noise_shape=None, seed=seed))
#     model.add(Dense(16, activation='relu', name = "Intermediate_1"))
#     Dropout(rate = 0.5, noise_shape=None, seed=seed)
#     # model.add(Dense(16, activation='relu', name = "Intermediate_2"))
#     model.add(Dense(1, activation='sigmoid', name = "Final_Output"))
#     optimizer = Adam(lr = 1e-3)
#     model.compile(loss='binary_crossentropy',
#                   optimizer=optimizer,
#                   metrics=['acc'])
#     # model.summary()

#     return model

In [ ]:
def create_model(lstm1, lstm2, dense1, maxlen):
    #Creating the network
    # embedding_size = 256
    # num_words = num_words # We chose this value earlier (10,000)
    # max_tokens = max_tokens # This values is also chosen from before (30)

    # For reproducability setting a seed
    seed = 42
    # np.random.seed(seed)

    model = Sequential()
    model.add(Embedding(input_dim=vocab_size,
                        output_dim=embedding_dim,
                        weights = [embedding_matrix], #Glove English
                        trainable = True,
                        input_length=maxlen,
                        name='layer_embedding'))
    model.add(Bidirectional(LSTM(units = lstm1, return_sequences=True, name = "BiLSTM_1")))
    model.add(Dropout(0.5, noise_shape=None, seed=seed))
    model.add(Bidirectional(LSTM(units = lstm2, return_sequences=False, name = "BiLSTM_2")))
    model.add(Dropout(0.5, noise_shape=None, seed=seed))
    model.add(Dense(dense1, activation='relu', name = "Intermediate_1"))
    Dropout(rate = 0.5, noise_shape=None, seed=seed)
    # model.add(Dense(16, activation='relu', name = "Intermediate_2"))
    model.add(Dense(1, activation='sigmoid', name = "Final_Output"))
    optimizer = Adam(lr = 1e-3)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['acc'])
    # model.summary()

    return model

In [ ]:
param_grid = dict(lstm1=[4,8,16, 32],
                  lstm2=[4, 8,16, 32],
                  dense1=[4, 8,16, 32],          
                  # vocab_size=[5000], 
                  # embedding_dim=[50, 30, 80],
                  maxlen=[29,50])

# param_grid = dict(lstm1=[16, 32],
#                   lstm2=[16, 32],
#                   dense1=[16, 32],          
#                   vocab_size=[5000], 
#                   embedding_dim=[50],
#                   maxlen=[29],
#                    random_state=1, refit=True,
#                    return_train_score=True
#                   )


# param_grid = dict(batch_size_param=[16, 32, 64, 128],
#                   lr_param=[1e-3, 5e-5, 3e-5, 2e-5],          
#                   vocab_size=[5000], 
#                   embedding_dim=[50],
#                   maxlen=[29],       
#                    )


# param_grid = dict(num_filters=[32, 64, 128],
#                   kernel_size=[3, 5, 7],
#                   vocab_size=[vocab_size],
#                   embedding_dim=[embedding_dim],
#                   maxlen=[maxlen])

In [ ]:
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 5)
# model = KerasClassifier(build_fn=create_model,
#                         epochs=3, batch_size= 64,
#                         verbose=False,
#                         validation_data=(X_test, test_target))
#                         ,
#                         callbacks=[es])

In [ ]:
import numpy as np

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath, encoding="latin-1") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [ ]:
embedding_dim = 50
# str = unicode(str, errors='ignore')
embedding_matrix = create_embedding_matrix(
    'Cow/big/cow-big.txt',   
    tokenizer.word_index, embedding_dim)

# with open(path, 'rb') as f:
#   contents = f.read()


In [ ]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size


0.9635591325732171

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

# Main settings
# epochs = 20
# embedding_dim = 50
# maxlen = 100

output_file = 'data/output.txt'

# Run grid search for each source (yelp, amazon, imdb)
# for source, frame in df.groupby('source'):
#     print('Running grid search for data set :', source)
#     sentences = df['sentence'].values
#     y = df['label'].values

# # Train-test split
# sentences_train, sentences_test, y_train, y_test = train_test_split(
#     sentences, y, test_size=0.25, random_state=1000)

# # Tokenize words
# tokenizer = Tokenizer(num_words=5000)
# tokenizer.fit_on_texts(sentences_train)
# X_train = tokenizer.texts_to_sequences(sentences_train)
# X_test = tokenizer.texts_to_sequences(sentences_test)

# # Adding 1 because of reserved 0 index
# vocab_size = len(tokenizer.word_index) + 1

# # Pad sequences with zeros
# X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
# X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# # Parameter grid for grid search
# param_grid = dict(num_filters=[32, 64, 128],
#                   kernel_size=[3, 5, 7],
#                   vocab_size=[vocab_size],
#                   embedding_dim=[embedding_dim],
#                   maxlen=[maxlen])
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import fbeta_score, make_scorer

# def my_custom_loss_func(y_true, y_pred):
#     diff = np.abs(y_true - y_pred).max()
#     return np.log1p(diff)

# score = make_scorer(my_custom_loss_func, greater_is_better=False)
epochs = 15 
cv_folds = 4
n_iter = 30
batch_size = 64

model = KerasClassifier(build_fn=create_model,
                        epochs=epochs, batch_size= batch_size,
                        verbose=False
                       )

grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                          cv= cv_folds, verbose=1, n_iter= n_iter,
                          # scoring= score,
                          return_train_score = True,
                          n_jobs = -1,
                          refit = True
                          )


# model = KerasClassifier(build_fn=create_model,
#                         epochs=15, batch_size= 128,
#                         verbose=False
#                        )

# grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
#                           cv=4, verbose=1, n_iter=10,
#                           # scoring= score,
#                           return_train_score = True,
                    
#                           n_jobs = -1
#                           )



from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping
early_stopping = EarlyStopping(monitor='loss', patience=5)


grid_result = grid.fit(x_train_pad, train_target, callbacks=[early_stopping])

# Evaluate testing set
test_accuracy = grid.score(x_test_pad, test_target)

# # Save and evaluate results
# prompt = input(f'finished {source}; write to file and proceed? [y/n]')
# if prompt.lower() not in {'y', 'true', 'yes'}:
#     break
# with open(output_file, 'a') as f:

    # s = ('Running {} data set\nBest Accuracy : '
    #       '{:.4f}\n{}\nTest Accuracy : {:.4f}\n\n')
    # output_string = s.format(
    #     source,
    #     grid_result.best_score_,
    #     grid_result.best_params_,
    #     test_accuracy)
    # print(output_string)
    # f.write(output_string)
print(grid_result.best_score_)
print(grid_result.best_params_)
# print(test_accuracy)
print(grid_result)

Fitting 4 folds for each of 30 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 16.5min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed: 43.1min finished


0.8035714328289032
{'maxlen': 50, 'lstm2': 32, 'lstm1': 8, 'dense1': 32}
RandomizedSearchCV(cv=4, error_score=nan,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7fd412544748>,
                   iid='deprecated', n_iter=30, n_jobs=-1,
                   param_distributions={'dense1': [4, 8, 16, 32],
                                        'lstm1': [4, 8, 16, 32],
                                        'lstm2': [4, 8, 16, 32],
                                        'maxlen': [29, 50]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=True, scoring=None, verbose=1)


In [ ]:
df_end = pd.DataFrame(grid_result.cv_results_)
df_end
# df_end.to_csv('Exp36_RNDSCV.csv')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_maxlen,param_lstm2,param_lstm1,param_dense1,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,mean_train_score,std_train_score
0,43.830024,0.181983,2.466766,0.041527,29,16,32,4,"{'maxlen': 29, 'lstm2': 16, 'lstm1': 32, 'dens...",0.773214,0.775000,0.826786,0.728571,0.775893,0.034776,27,0.992262,0.992262,0.987500,0.989286,0.990327,0.002035
1,38.571409,0.428240,2.561585,0.192288,50,8,16,32,"{'maxlen': 50, 'lstm2': 8, 'lstm1': 16, 'dense...",0.771429,0.785714,0.817857,0.766071,0.785268,0.020139,17,0.990476,0.992262,0.988690,0.991667,0.990774,0.001364
2,37.750784,0.290465,2.533469,0.102077,50,16,8,32,"{'maxlen': 50, 'lstm2': 16, 'lstm1': 8, 'dense...",0.776786,0.769643,0.798214,0.751786,0.774107,0.016632,28,0.996429,0.992262,0.988095,0.991667,0.992113,0.002958
3,37.471072,0.356692,2.522947,0.106315,50,8,8,8,"{'maxlen': 50, 'lstm2': 8, 'lstm1': 8, 'dense1...",0.780357,0.814286,0.816071,0.755357,0.791518,0.025266,6,0.991667,0.992262,0.987500,0.989881,0.990327,0.001853
4,48.135143,0.493261,2.458181,0.069341,29,32,32,16,"{'maxlen': 29, 'lstm2': 32, 'lstm1': 32, 'dens...",0.792857,0.798214,0.794643,0.767857,0.788393,0.012012,9,0.996429,0.992262,0.987500,0.991071,0.991815,0.003188
5,41.912752,0.559635,2.476370,0.071566,50,32,16,8,"{'maxlen': 50, 'lstm2': 32, 'lstm1': 16, 'dens...",0.769643,0.810714,0.798214,0.732143,0.777679,0.030212,25,0.996429,0.992262,0.986310,0.991071,0.991518,0.003605
6,36.786686,0.328135,2.444383,0.077370,29,8,8,8,"{'maxlen': 29, 'lstm2': 8, 'lstm1': 8, 'dense1...",0.800000,0.816071,0.798214,0.775000,0.797321,0.014644,2,0.989286,0.991071,0.986310,0.990476,0.989286,0.001835
7,38.301624,2.046470,2.513177,0.115102,50,8,8,32,"{'maxlen': 50, 'lstm2': 8, 'lstm1': 8, 'dense1...",0.769643,0.798214,0.803571,0.755357,0.781696,0.019940,22,0.995238,0.992262,0.987500,0.991071,0.991518,0.002772
8,34.762803,0.251867,2.564165,0.057762,29,4,4,16,"{'maxlen': 29, 'lstm2': 4, 'lstm1': 4, 'dense1...",0.783929,0.825000,0.778571,0.744643,0.783036,0.028530,19,0.994048,0.986310,0.986310,0.985119,0.987946,0.003556
9,36.295340,0.186138,2.502817,0.134308,50,16,4,16,"{'maxlen': 50, 'lstm2': 16, 'lstm1': 4, 'dense...",0.773214,0.796429,0.803571,0.737500,0.777679,0.025769,25,0.995238,0.992262,0.988095,0.988095,0.990923,0.003017


In [ ]:
print(grid_result.best_score_)
print(grid_result.best_params_)

# test_accuracy = grid.score(x_test_pad, test_target)
# print(test_accuracy)
print(grid_result)


# train_accuracy = grid.score(x_train_pad, train_target)
# print(train_accuracy)

0.8035714328289032
{'maxlen': 50, 'lstm2': 32, 'lstm1': 8, 'dense1': 32}
RandomizedSearchCV(cv=4, error_score=nan,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7fd412544748>,
                   iid='deprecated', n_iter=30, n_jobs=-1,
                   param_distributions={'dense1': [4, 8, 16, 32],
                                        'lstm1': [4, 8, 16, 32],
                                        'lstm2': [4, 8, 16, 32],
                                        'maxlen': [29, 50]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=True, scoring=None, verbose=1)


In [ ]:
threshold = 0.5
predicted = grid.predict(x_train_pad)

predicted = np.where(predicted > threshold, 1,0)


predicted  = np.reshape(predicted, len(predicted))


eval_predicted = grid.predict(x_test_pad)

eval_predicted = np.where(eval_predicted > threshold, 1,0)

eval_predicted  = np.reshape(eval_predicted, len(eval_predicted))

# train acc

no_predictions = len(predicted)
print(no_predictions)

right = np.sum(predicted == train_target)

training_accuracy = right/ no_predictions
print("Training Accuracy: ", training_accuracy)


# test acc

no_predictions_eval = len(eval_predicted)
print(no_predictions_eval)

right_eval = np.sum(eval_predicted == test_target)

test_accuracy = right_eval/ no_predictions_eval
print("Test Accuracy: ", test_accuracy)

2240
Training Accuracy:  0.9901785714285715
720
Test Accuracy:  0.7652777777777777


In [ ]:
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as sk

# loss, accuracy = grid.score(x_train_pad, train_target)
# print("Training Accuracy: {:.4f}".format(accuracy))
# print("Training Loss: {:.4f}".format(loss))


precision = sk.precision_score(train_target, predicted)
print("Training Precision: ", precision)
recall = sk.recall_score(train_target, predicted)
print("Training recall: ", recall)
roc_auc = sk.roc_auc_score(train_target, predicted)
print("Training roc_auc: ", roc_auc)
f1 = sk.f1_score(train_target, predicted)
print("Training f1: ", f1)


# loss, accuracy = grid.score(x_test_pad, test_target)
# print("Eval Accuracy: {:.4f}".format(accuracy))
# print("Eval Loss: {:.4f}".format(loss))


precision = sk.precision_score(test_target, eval_predicted)
print("Eval Precision: ", precision)
recall = sk.recall_score(test_target, eval_predicted)
print("Eval recall: ", recall)
roc_auc = sk.roc_auc_score(test_target, eval_predicted)
print("Eval roc_auc: ", roc_auc)
f1 = sk.f1_score(test_target, eval_predicted)
print("Eval f1: ", f1)

Training Precision:  0.9949290060851927
Training recall:  0.9829659318637275
Training roc_auc:  0.9894700834841987
Training f1:  0.9889112903225806
Eval Precision:  0.6824817518248175
Eval recall:  0.6951672862453532
Eval roc_auc:  0.7511313149630314
Eval f1:  0.6887661141804788


In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [ ]:
plot_history(history) 

NameError: ignored

In [ ]:
# Check the model
result = model.evaluate(x_test_pad, test_target, verbose = 0)
print("Accuracy: {0:.2%}".format(result[1]))

In [ ]:
# serialize model to JSON and save it
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
#To load the model back in:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

In [ ]:
# Verify if it loaded properly
loaded_model.compile(loss='binary_crossentropy', 
                     optimizer=Adam(lr = 1e-3),
                     metrics=['accuracy'])
score = loaded_model.evaluate(x_test_pad, test_target, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
# IT WORKS!!!

In [ ]:
# To reverse tokinized maps
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))


def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [ ]:
print(tokens_to_string(x_test_pad[0]))
print(tokens_to_string(x_test_pad[1]))
print(tokens_to_string(x_test_pad[2]))

In [ ]:
print(x_test_pad[2:5])
print()
print(tokens_to_string(x_test_pad[2]))
print()
print(tokens_to_string(x_test_pad[3]))
print()
print(tokens_to_string(x_test_pad[4]))
print()
loaded_model.predict(x_test_pad[2:5])
# The example seems to be quite good (0:3 are skipped, because they all give value of 1)